Careless responder feature engineering and analysis (focus on engage_psycap survey only)

In [ ]:
#getting and working with data
import pandas as pd
import numpy as np
import re
import os
from itertools import groupby
import datetime as dt
from numpy import median

#visualizing results
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt
#import yellowbrick as yb

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

import warnings; warnings.simplefilter('ignore')
np.set_printoptions(suppress=True)

In [ ]:
data_path = 'C:/Users/Schindler/Documents/Schindler_Lab/Data/Clinical projects/TILES/final_data/data_final_combined.pkl'

In [ ]:
#read in csv containing participant info
data = pd.read_pickle(data_path)
data = pd.DataFrame(data = data)
#data_PF.reset_index(inplace=True)

print('Original data shape:\n', data.shape, '\n')
#ensure no replicate ID (212 participants in study)
print('Original data unique IDs:\n', data['ParticipantID'].unique().shape, '\n')
#ensure no replicate ID (212 participants in study)
print('Original data unique IDs:\n', data['MitreID'].unique().shape, '\n')
#how much missing data is there?
print('Original data missing value counts:\n', data.isnull().sum(), '\n')
#what is the data type of each column?
print('Original data data types:\n', data.info(), '\n')

In [ ]:
data['survey_type'].value_counts()

In [ ]:
#should be 71
len(data['wave_study_day'].unique())

### notes for CR features for Engage

Should have answered every question

Longstring
- All questions use same scale (1=not at all, 7=very much), but there are 5 different constructs assessed
    - make column with longest string
    - make column with number that longest string consisted of

Semantic consistency
- Internal consistency (within construct) should be greater than consistency across constructs

Semantic antonyms
- Not applicable

Semantic synonyms 
- Hindrance stressors should be negatively correlated with support

In [ ]:
#split off completed PF and related columns
engage_psycap_only = data[(data['survey_type'] == 'engage_psycap') & (data['completed'] == 1.0)]


engage_psycap_only.shape
engage_psycap_only['ParticipantID'].unique().shape

In [ ]:
engage_psycap_only = engage_psycap_only[['index', 'MitreID', 'ParticipantID', 'PrimaryUnit', 'SmartPhone',
       'Sex', 'Shift', 'Wave', 'survey_id', 'survey_type', 'date_time',
       'date', 'time', 'Timesent', 'completed', 'start_delay',
       'time_to_complete', 'activity_num', 'location_num',
       'engage_location', 'engage_activity', 'engage_3', 'engage_4',
       'engage_5', 'engage_6', 'engage_7', 'engage_8', 'engage_9',
       'engage_10', 'engage_11', 'engage_12', 'engage_13', 'engage_14',
       'engage_15', 'engage_16', 'engage_17', 'engage_18', 'engage_19',
       'engage_20', 'engage_21', 'engage_22', 'engage_23', 'engage_24',
       'engage_25', 'engage_26', 'engage_27', 'engage_28', 'engage_29',
       'engage_mgt', 'psycap_mgt', 'support_mgt', 'challenge_mgt',
       'hindrance_mgt',
       'wave_study_date', 'wave_study_day',
       'survey_count', 'day_of_week', 'working', 'at_work']]

In [ ]:
#create study date bins
engage_psycap_only['wave_study_date_bin'] = pd.cut(engage_psycap_only['wave_study_day'], 5)
engage_psycap_only['wave_study_date_bin'].value_counts()

In [ ]:
#create feature that is count of missing values from survey
engage_psycap_only.loc[engage_psycap_only.loc[:, 'engage_location':'engage_29'].isnull().sum(axis=1) > 0, 'missing_values'] = engage_psycap_only.loc[:, 'engage_location':'engage_29'].isnull().sum(axis=1)
engage_psycap_only.loc[engage_psycap_only.loc[:, 'engage_location':'engage_29'].isnull().sum(axis=1) == 0, 'missing_values'] = 0.0

In [ ]:
#create feature that is number of surveys that were not completed

part_id = engage_psycap_only['ParticipantID'].unique()

for participant in part_id:
    missing = 19 - engage_psycap_only[engage_psycap_only['ParticipantID'] == participant]['survey_count'].max()
    engage_psycap_only.loc[(engage_psycap_only['ParticipantID'] == participant), 'uncompleted'] = missing

In [ ]:
#create feature that is number of surveys that were not completed within wave study date bin

part_id = engage_psycap_only['ParticipantID'].unique()
day_bins = engage_psycap_only['wave_study_date_bin'].unique()

for participant in part_id:
    for unique_bin in day_bins:
        missing = 6 - engage_psycap_only.loc[(engage_psycap_only['ParticipantID'] == participant) & (engage_psycap_only['wave_study_date_bin'] == unique_bin)].shape[0]
        if missing < 0:
            missing = 0
        engage_psycap_only.loc[(engage_psycap_only['ParticipantID'] == participant) & (engage_psycap_only['wave_study_date_bin'] == unique_bin), 'uncompleted_day_bin'] = missing

In [ ]:
#long string analysis (e.g. max length of same number answered for 'engage_location':'engage_29')
#create features related to long string analysis (feature of how long the string is and feature of what the string consisted of)

max_strings = []
max_answers = []

for index, row in engage_psycap_only.iterrows():
    
    groups = groupby(row['engage_location':'engage_29'])
    result = [(label, sum(1 for _ in group)) for label, group in groups]

    max_pair = max(result, key=lambda x:x[1])
    max_string_length = max_pair[1]
    max_answer = max_pair[0]

    max_strings.append(max_string_length)
    
    max_answers.append(max_answer)

engage_psycap_only['longest_string_count'] = max_strings
engage_psycap_only['longest_string_answer'] = max_answers

In [ ]:
#create feature that is ratio of average longstring during first day bin to average longstring during last time bin

part_id = engage_psycap_only['ParticipantID'].unique()
day_bins = engage_psycap_only['wave_study_date_bin'].unique()

for participant in part_id:
    string_1 = engage_psycap_only.loc[(engage_psycap_only['ParticipantID'] == participant) & (engage_psycap_only['wave_study_date_bin'] == day_bins[0])]['longest_string_count'].mean() 
    string_2 = engage_psycap_only.loc[(engage_psycap_only['ParticipantID'] == participant) & (engage_psycap_only['wave_study_date_bin'] == day_bins[-1])]['longest_string_count'].mean()

    engage_psycap_only.loc[(engage_psycap_only['ParticipantID'] == participant), 'longest_string_ratio'] = string_2 / string_1

In [ ]:
#create feature that is ratio of average longstring during current bin to average longstring during last time bin

part_id = engage_psycap_only['ParticipantID'].unique()
day_bins = engage_psycap_only['wave_study_date_bin'].unique()

for participant in part_id:
    for unique_bin in day_bins:
        string_1 = engage_psycap_only.loc[(engage_psycap_only['ParticipantID'] == participant) & (engage_psycap_only['wave_study_date_bin'] == unique_bin)]['longest_string_count'].mean() 
        string_2 = engage_psycap_only.loc[(engage_psycap_only['ParticipantID'] == participant) & (engage_psycap_only['wave_study_date_bin'] == day_bins[-1])]['longest_string_count'].mean()

        engage_psycap_only.loc[(engage_psycap_only['ParticipantID'] == participant) & (engage_psycap_only['wave_study_date_bin'] == unique_bin), 'longest_string_ratio_bin'] = string_2 / string_1

In [ ]:
#create feature that is ratio of number of longstrings over 7 to total number of surveys taken

part_id = engage_psycap_only['ParticipantID'].unique()

for participant in part_id:
    count_over_7 = engage_psycap_only.loc[(engage_psycap_only['ParticipantID'] == participant) & (engage_psycap_only['longest_string_count'] >= 13)].shape[0]
    
    engage_psycap_only.loc[(engage_psycap_only['ParticipantID'] == participant), 'ls_ratio'] = count_over_7 / engage_psycap_only.loc[(engage_psycap_only['ParticipantID'] == participant)].shape[0]

In [ ]:
#create feature that is ratio of number of longstrings over 7 during current bin to total number of surveys taken for that bin

part_id = engage_psycap_only['ParticipantID'].unique()
day_bins = engage_psycap_only['wave_study_date_bin'].unique()

for participant in part_id:
    for unique_bin in day_bins:
        count_over_7 = engage_psycap_only.loc[(engage_psycap_only['ParticipantID'] == participant) 
                                           & (engage_psycap_only['longest_string_count'] >= 13) 
                                           & (engage_psycap_only['wave_study_date_bin'] == unique_bin)].shape[0]
        try:
            engage_psycap_only.loc[(engage_psycap_only['ParticipantID'] == participant) & (engage_psycap_only['wave_study_date_bin'] == unique_bin), 'ls_ratio_bin'] = count_over_7 / engage_psycap_only.loc[(engage_psycap_only['ParticipantID'] == participant) & (engage_psycap_only['wave_study_date_bin'] == unique_bin)].shape[0]
        except:
            engage_psycap_only.loc[(engage_psycap_only['ParticipantID'] == participant) & (engage_psycap_only['wave_study_date_bin'] == unique_bin), 'ls_ratio_bin'] = np.nan 

In [ ]:
#create feature that is binary for if time_to_complete is very short or long (0 is neither)
bad_times_low = []
bad_times_high = []

for index, row in engage_psycap_only.iterrows():
    
    if (row['time_to_complete'] <= 20): 
        bad_times_low.append(1)
    else:
        bad_times_low.append(0)
    
    if (row['time_to_complete'] > 1200): 
        bad_times_high.append(1)
    else:
        bad_times_high.append(0)

engage_psycap_only['bad_times_low'] = bad_times_low
engage_psycap_only['bad_times_high'] = bad_times_high

In [ ]:
engage_psycap_CR = engage_psycap_only[['MitreID', 'ParticipantID', 'PrimaryUnit', 'SmartPhone',
       'Sex', 'Shift', 'Wave', 'survey_type', 'start_delay',
       'time_to_complete', 'activity_num', 'location_num',
       'engage_location', 'engage_activity', 'engage_mgt', 'psycap_mgt', 'support_mgt', 'challenge_mgt',
       'hindrance_mgt',
       'wave_study_day', 'survey_count', 'day_of_week', 'working',
       'wave_study_date_bin', 'missing_values', 'uncompleted',
       'uncompleted_day_bin', 'longest_string_count',
       'longest_string_answer', 'longest_string_ratio',
       'longest_string_ratio_bin', 'bad_times_low', 'bad_times_high', 'ls_ratio', 'ls_ratio_bin']]

engage_psycap_CR.head()

In [ ]:
path_demog_prepost = 'C:/Users/Schindler/Documents/Schindler_Lab/Data/Clinical projects/TILES/final_data/Demog, PRE, PST survey composites.csv'

In [ ]:
#read in data_demog_prepost, contains demographic information and pre/post questionaires
data_demog_prepost = pd.read_csv(path_demog_prepost)
data_demog_prepost = pd.DataFrame(data = data_demog_prepost)
print('Original demog_prepost shape:\n', data_demog_prepost.shape, '\n')
#replace blnaks with nans
data_demog_prepost = data_demog_prepost.replace(' ', np.nan)
#ensure no replicate ID (eg one row per participant in study)
print('Original demog_prepost unique IDs:\n', data_demog_prepost['ID'].unique().shape, '\n')
print('Original demog_prepost missing value couts:\n', data_demog_prepost.isnull().sum(), '\n')
print('Original demog_prepost data types:\n', data_demog_prepost.info(), '\n')
data_demog_prepost.head()

In [ ]:
#add the information contained in data_demog_prepost to psych_flex_CR
#first create new data table of data_demog_prepost that contains the same number of rows for each participant in that is in psych_flex_CR
#(e.g. replicate data_demog_prepost so same length as psych_flex_CR for each participant)

participants = engage_psycap_CR['MitreID'].unique()

engage_CR_demog = pd.DataFrame()

for part in participants:
    df_part_long = pd.concat([data_demog_prepost[data_demog_prepost['ID'] == part]]*len(engage_psycap_CR[engage_psycap_CR['MitreID'] ==  part]), ignore_index=True)
    df_part_long.reset_index(inplace=True)
    engage_CR_demog_int = pd.concat([df_part_long, engage_psycap_CR[engage_psycap_CR['MitreID'] ==  part].reset_index()], axis = 1)
    engage_CR_demog = engage_CR_demog.append(engage_CR_demog_int)

#confirm the two data tables are now the same lenght
print('psych_flex_CR and psych_flex_CR_demog are the same length:', engage_psycap_CR.shape[0] == engage_CR_demog.shape[0])
print(engage_psycap_CR.shape[0])
print(engage_CR_demog.shape[0])
print('does the math make sense?', engage_CR_demog.shape[0] == (engage_CR_demog['ID'].values == engage_CR_demog['MitreID'].values).sum())
engage_CR_demog.head()

In [ ]:
engage_CR_demog_features = ['MitreID', 'ParticipantID',
       'PrimaryUnit', 'SmartPhone', 'Sex', 'Shift', 'Wave', 'bornUS', 'lang', 'englyrs', 
                       'activity_num', 'location_num',
       'engage_location', 'engage_activity', 'engage_mgt', 'psycap_mgt', 'support_mgt', 'challenge_mgt',
       'hindrance_mgt', 'ocb', 'inter.deviance', 'org.deviance', 'extraversion',
       'agreeableness', 'conscientiousness', 'neuroticism', 'openness',
       'psqi', 'fatigue', 'Stress', 
       'Flexibility', 'Inflexibility', 'Engagement', 'start_delay',
       'time_to_complete', 'wave_study_day', 'survey_count', 'day_of_week',
       'working', 'wave_study_date_bin', 'missing_values', 'uncompleted',
       'uncompleted_day_bin', 'longest_string_count',
       'longest_string_answer', 'longest_string_ratio',
       'longest_string_ratio_bin', 'bad_times_low', 'bad_times_high',
       'ls_ratio', 'ls_ratio_bin']


engage_CR_demog_df = engage_CR_demog[engage_CR_demog_features]

In [ ]:
#save to csv
engage_CR_demog_df.to_csv('engage_CR_demog_df.csv')

#pickle to save
engage_CR_demog_df.to_pickle('engage_CR_demog_df.pkl')

In [ ]:
engage_CR_demog_df.groupby('wave_study_date_bin').corr().to_csv('corr.csv')

In [ ]:
g = sns.catplot(x="wave_study_date_bin", y="longest_string_count", data=engage_CR_demog_df, kind="bar")
g.fig.set_size_inches(15,5)

In [ ]:
engage_CR_demog_df['longest_string_answer'] = engage_CR_demog_df['longest_string_answer'].astype(float)
g = sns.catplot(x="wave_study_date_bin", y="longest_string_answer", data=engage_CR_demog_df, kind="bar")
g.fig.set_size_inches(15,5)

In [ ]:
#use order from above
g = sns.catplot(x="wave_study_date_bin", y="longest_string_ratio_bin", data=engage_CR_demog_df, kind="bar")
g.fig.set_size_inches(15,5)

In [ ]:
#use order from above
g = sns.catplot(x="wave_study_date_bin", y="ls_7_ratio_bin", data=engage_CR_demog_df, kind="bar")
g.fig.set_size_inches(15,5)

In [ ]:
#use order from above
g = sns.catplot(x="longest_string_count", y="bad_times_high", data=engage_CR_demog_df, kind="bar", col='wave_study_date_bin', col_wrap=3)
g.fig.set_size_inches(15,5)

In [ ]:
#use order from above
g = sns.catplot(x="longest_string_count", y="ls_7_ratio", data=engage_CR_demog_df, kind="bar", col='wave_study_date_bin', col_wrap=3)
g.fig.set_size_inches(15,5)

In [ ]:
unique_ids = engage_CR_demog_df['MitreID'].unique()
for u_id in unique_ids:
    g = sns.catplot(x="wave_study_day", y="longest_string_count", data=engage_CR_demog_df[engage_CR_demog_df['MitreID'] == u_id], kind="bar")
    g.fig.set_size_inches(10,5)
    #axes = g.axes
    plt.ylim([0,30])
    #g.set_ylim(0, 13)
    plt.show()

In [ ]:
engage_CR_demog_df[engage_CR_demog_df['longest_string_count'] > 0].shape

In [ ]:
#get participant ID order when sorted by longest string in the last time bin
wave_1_ls = engage_CR_demog_df[engage_CR_demog_df['wave_study_date_bin'] == engage_CR_demog_df['wave_study_date_bin'].unique()[-1]].sort_values('longest_string_count')
ls_index = wave_1_ls['MitreID'].unique()

In [ ]:
#use order from above
g = sns.catplot(x="MitreID", y="longest_string_count", data=engage_CR_demog_df, kind="bar", col='wave_study_date_bin', 
                ci=None, order=ls_index)
g.fig.set_size_inches(25,10)

In [ ]:
#use order from above
g = sns.catplot(x="MitreID", y="ls_7_ratio_bin", data=engage_CR_demog_df, kind="bar", col='wave_study_date_bin', 
                ci=None, order=ls_index)
g.fig.set_size_inches(25,10)

In [ ]:
#use order from above
g = sns.catplot(x="MitreID", y="longest_string_ratio_bin", data=engage_CR_demog_df, kind="bar", col='wave_study_date_bin', 
                ci=None, order=ls_index)
g.fig.set_size_inches(25,10)

In [ ]:
g = sns.catplot(x="bad_times_high", y="longest_string_count", data=engage_CR_demog_df, kind="bar", col='wave_study_date_bin')
g.fig.set_size_inches(25,10)

In [ ]:
g = sns.catplot(x="fatigue", y="longest_string_count", data=engage_CR_demog_df, kind="bar", col='wave_study_date_bin', order=['0', '10', '20', '30', '40', '50', '60', '70', '80', '90', '100'])
g.fig.set_size_inches(25,10)

In [ ]:
g = sns.FacetGrid(engage_CR_demog_df, col="wave_study_date_bin")
g.map(sns.regplot, "ocb", "longest_string_count")
g.fig.set_size_inches(25,10)



In [ ]:
g = sns.FacetGrid(engage_CR_demog_df, col="wave_study_date_bin")
g.map(sns.regplot, "org.deviance", "longest_string_count")
g.fig.set_size_inches(25,10)



In [ ]:
g = sns.FacetGrid(engage_CR_demog_df, col="wave_study_date_bin")
g.map(sns.regplot, "inter.deviance", "longest_string_count")
g.fig.set_size_inches(25,10)



In [ ]:
engage_CR_demog_df['Flexibility'] = engage_CR_demog_df['Flexibility'].astype(float)

In [ ]:
g = sns.FacetGrid(engage_CR_demog_df, col="wave_study_date_bin")
g.map(sns.regplot, "Flexibility", "longest_string_count")
g.fig.set_size_inches(25,10)



In [ ]:
engage_CR_demog_df['Stress'] = engage_CR_demog_df['Stress'].astype(float)

In [ ]:
g = sns.FacetGrid(engage_CR_demog_df, col="wave_study_date_bin")
g.map(sns.regplot, "Stress", "longest_string_count")
g.fig.set_size_inches(25,10)

